### 국내 판매중인 중고차/신차 모델명 크롤러
<hr>

#### 순서
> 1) 크롤링 실행할 URL 접속 
> 2) 데이터 크롤링 
> 3) 데이터프레임(테이블 형식) 변환
> 4) 데이터 정리
> 5) CSV로 변환 

#### 1. 크롤링 실행할 URL 접속

In [4]:
### selenium importing
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

### chrome driver
# windows
# driver = webdriver.Chrome('F:\python\house_crawling\chromedriver.exe')

# MacOS
# driver = webdriver.Chrome('/Users/wanya/Simple_Car_model_crawling/chromedriver')
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
  
# 크롤링할 Encar 사이트 접속
driver.get('http://www.encar.com/db/db_carsinfo.do?method=newpricePop')
delay = 5
driver.implicitly_wait(delay)
"""리소스(url 호출)를 기다리는 시간(3초)"""



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/wanya/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


'리소스(url 호출)를 기다리는 시간(3초)'

#### 2. 데이터 크롤링

In [5]:
'''
크롤링 대상 HTML Elemnet  ID 
1) 제조사   : //*[@id="mnfccd"]        | //*[@id="mnfccd"]
2) 모델    : //*[@id="mdlgroupcd"]  
3) 세부모델 : //*[@id="mdlcd"]         | //*[@id="mdlcd"]
4) 형식연도 : //*[@id="caryear"]       | //*[@id="caryear"]

크롤링 케이스 
1) 제조사 -> 모델    -> 세부모델 -> 형식연도
2) 제조사 -> 모델    -> 형식연도
3) 제조사 -> 세부모델 -> 형식연도 
'''
###################################################
# 크롤링 데이터를 담을 List
car_list = [[], [], [], []]
# 크롤링하지 않을 데이터 키워드
hide_keyword = ['제조사', '모델', '세부모델',' 형식연도']
###################################################

### 1) 제조사 선택 ###
select = Select(driver.find_element(By.ID, 'mnfccd'))

# select 태그의 [제조사] option 전체를 가져오는 loop
for opt in select.options:
    print("---파싱중인 제조사---")
    print(opt.text)
    # 제조사 선택
    select.select_by_visible_text(opt.text) 
    
    ### 2. 제조사 -> 모델 
    try:
        select_2 = Select(driver.find_element(By.ID, 'mdlgroupcd'))

        # select 태그의 [모델] option 전체를 가져오는 loop
        for opt_2 in select_2.options:
            # print(opt_2.text)
            # 모델 선택
            select_2.select_by_visible_text(opt_2.text)
            
            try:
                #### 3. 모델 -> 세부모델 선택
                select_3 = Select(driver.find_element(By.ID, 'mdlcd'))

                # select 태그의 [세부모델] option 전체를 가져오는 loop
                for opt_3 in select_3.options:
                    # print(opt_3.text)
                    # 세부모델 선택
                    select_3.select_by_visible_text(opt_3.text)
                    ### 4. 세부모델 -> 형식연도 선택
                    select_4 = Select(driver.find_element(By.ID, 'caryear'))
                    for opt_4 in select_4.options:
                        # print(opt_4.text)
                        check_opt_4 = opt_4.text not in hide_keyword
                        if check_opt_4:
                            # print(opt_4.text)
                            # 크롤링 케이스 1), '제조사, 모덷, 세부모델, 형식연도'
                            car_list.append([opt.text, opt_2.text, opt_3.text, opt_4.text])

            #### 3-1. 모델 -> 세부모델 선택 없을 때, 바로 형식연도 크롤링
            except Exception as E:
                #### 4. 모델 -> 형식연도 선택
                select_3 = Select(driver.find_element(By.ID, 'caryear'))

                # select 태그의 [형식연도] option 전체를 가져오는 loop
                for opt_3 in select_3.options:
                    # print(opt_3.text)
                    # 크롤링하지 않아도 될 데이터는 리스트 삽입에서 제외하는 조건문
                    check_opt_3= opt_3.text not in hide_keyword
                    if check_opt_3:
                        # print(opt_3.text)
                        # 크롤링 케이스 2), '제조사, 모덷, '모델', 형식연도'
                        car_list.append([opt.text, opt_2.text, opt_2.text , opt_3.text])

    ### 2-1. 제조사 -> 세부모델               
    except Exception as E:
        #### 3. 모델 -> 세부모델 선택
        select_2 = Select(driver.find_element(By.ID, 'mdlcd'))

        # select 태그의 [세부모델] option 전체를 가져오는 loop
        for opt_2 in select_2.options:
            # print(opt_2.text)
            select_2.select_by_visible_text(opt_2.text)
            
            ### 4. 세부모델 -> 형식연도 선택
            select_3 = Select(driver.find_element(By.ID, 'caryear'))

            # select 태그의 [형식연도] option 전체를 가져오는 loop
            for opt_3 in select_3.options:
                # print(opt_3.text)
                check_opt_3= opt_3.text not in hide_keyword
                if check_opt_3:
                    # print(opt_3.text)
                    # 크롤링 케이스 3), '제조사, 세부모덷, '세부모델', 형식연도'
                    car_list.append([opt.text, opt_2.text, opt_2.text, opt_3.text])
                    # select_3.select_by_visible_text(opt_3.text)

### 참고 메소드 ###
# select by visible text
# select.select_by_visible_text('제네시스')
# select = Select(driver.find_element_by_id('mdlgroupcd'))

# # select by value 
# select.select_by_visible_text('G70')

---파싱중인 제조사---
제조사
---파싱중인 제조사---
현대
---파싱중인 제조사---
제네시스
---파싱중인 제조사---
기아
---파싱중인 제조사---
쉐보레(GM대우)
---파싱중인 제조사---
르노코리아(삼성)
---파싱중인 제조사---
쌍용
---파싱중인 제조사---
BMW
---파싱중인 제조사---
닛산
---파싱중인 제조사---
닷지
---파싱중인 제조사---
도요타
---파싱중인 제조사---
람보르기니
---파싱중인 제조사---
랜드로버
---파싱중인 제조사---
렉서스
---파싱중인 제조사---
로터스
---파싱중인 제조사---
롤스로이스
---파싱중인 제조사---
링컨
---파싱중인 제조사---
마세라티
---파싱중인 제조사---
마이바흐
---파싱중인 제조사---
맥라렌
---파싱중인 제조사---
머큐리
---파싱중인 제조사---
미니
---파싱중인 제조사---
미쯔비시
---파싱중인 제조사---
벤츠
---파싱중인 제조사---
벤틀리
---파싱중인 제조사---
볼보
---파싱중인 제조사---
뷰익
---파싱중인 제조사---
사브
---파싱중인 제조사---
쉐보레
---파싱중인 제조사---
스마트
---파싱중인 제조사---
스바루
---파싱중인 제조사---
스즈키
---파싱중인 제조사---
시트로엥
---파싱중인 제조사---
아우디
---파싱중인 제조사---
애스턴마틴
---파싱중인 제조사---
인피니티
---파싱중인 제조사---
재규어
---파싱중인 제조사---
지프
---파싱중인 제조사---
캐딜락
---파싱중인 제조사---
크라이슬러
---파싱중인 제조사---
페라리
---파싱중인 제조사---
포드
---파싱중인 제조사---
포르쉐
---파싱중인 제조사---
폭스바겐
---파싱중인 제조사---
폰티악
---파싱중인 제조사---
폴스타
---파싱중인 제조사---
푸조
---파싱중인 제조사---
피아트
---파싱중인 제조사---
혼다


In [6]:
# 결과값 list length 확인
print(len(car_list))

9463


#### 3. 데이터 프레임(테이블 형태)변환

In [7]:
import pandas as pd
# 테이블 컬럼이름 지정 (제조사:maker, 모델명:model, 세부모델:model_detail, 형식연도:model_year)
df = pd.DataFrame(car_list, columns=['maker', 'model', 'model_detail', 'model_year'])
# 데이터 1~1000개 조회
df.head(1000)

,maker,model,model_detail,model_year
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,제조사,모델,세부모델,연식
...,...,...,...,...
995,현대,제네시스,제네시스,2012
996,현대,제네시스,제네시스,2011
997,현대,제네시스,제네시스,2010
998,현대,제네시스,제네시스,2009


#### 4. 데이터 정리

In [8]:
# 결측(없는, 누락) 데이터 확인
df.isnull().sum()

maker           4
model           4
model_detail    4
model_year      4
dtype: int64

In [9]:
# 결측데이터 제거 후 확인
df_1 = df.dropna()
df_1.isnull().sum()

maker           0
model           0
model_detail    0
model_year      0
dtype: int64

In [10]:
# 각 컬럼중에 제조사/모델/세부모델/형식연도 의 값을 가진 튜플을 제거하기 위한 인덱스 설정
index = df_1[(df['maker'] == '제조사') | (df['model'] == '모델') | (df['model_detail'] == '세부모델') | (df['model_year'] == '형식연도')].index

# 위에 해당되는 인덱스 데이터 제거
df_2 = df_1.drop(index)

# 데이터 확인
df_2

/var/folders/f8/11bx78sd5gz2q2ppw4079pjc0000gn/T/ipykernel_3054/739039834.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = df_1[(df['maker'] == '제조사') | (df['model'] == '모델') | (df['model_detail'] == '세부모델') | (df['model_year'] == '형식연도')].index


,maker,model,model_detail,model_year
37,현대,i30,i30 (PD),2020
38,현대,i30,i30 (PD),2019
39,현대,i30,i30 (PD),2018
40,현대,i30,i30 (PD),2017
41,현대,i30,i30 (PD),2016
...,...,...,...,...
9458,혼다,시빅,시빅,2010
9459,혼다,시빅,시빅,2009
9460,혼다,시빅,시빅,2008
9461,혼다,시빅,시빅,2007


#

#### 5. 데이터 프레임(테이블) -> CSV 로 변환
* 자동화??
    - 바로 dataframe -> db insert

In [11]:
df_2.to_csv('encar_list_new.csv', sep=',', na_rep='') 